In [82]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
# import bert
# from bert import BertModelLayer

from tensorflow import keras
import tensorflow as tf
import os
import sentencepiece as spm
import tf_sentencepiece as tfs
import sys
sys.path.extend(["/var/extra/users/jgeorge/tf2.0/git/models/"])


import json
import math
import os
import csv
import collections
import pandas as pd

from absl import app
from absl import flags
from absl import logging
import tensorflow as tf
import tensorflow_hub as hub

# pylint: disable=g-import-not-at-top,redefined-outer-name,reimported
from official.modeling import model_training_utils
from official.nlp import bert_modeling as modeling
from official.nlp import bert_models
from official.nlp import optimization
from official.nlp.bert import common_flags
from official.nlp.bert import input_pipeline
from official.nlp.bert import model_saving_utils
from official.utils.misc import distribution_utils
from official.utils.misc import keras_utils
from official.nlp.bert import tokenization

from sklearn.model_selection import train_test_split

In [83]:
from tensorflow.python.keras.engine import network

In [84]:
print(tf.__version__)

2.1.0


In [85]:
tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:4', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:5', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:6', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:7', device_type='GPU')]

In [86]:
##restricting no of gpus
gpus = tf.config.experimental.list_physical_devices('GPU')
device_to_use = gpus[2]
tf.config.experimental.set_memory_growth(device_to_use,True)
tf.config.experimental.set_visible_devices(device_to_use, 'GPU')
print(tf.config.experimental.get_visible_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU')]


In [87]:
dish_data_path='/var/extra/users/jgeorge/tf2.0/input/dish/data/jan17_2020/dishDataJan17.txt'
df = pd.read_csv(dish_data_path,sep='\t',header=None,names=['filename','text','granular_intent','ru_intent'])

In [88]:
df.head()

,filename,text,granular_intent,ru_intent
0,INT-sv1appis14-1504137880316-305603_4567,can you send my bill to my mail?,billing-preferences,billing-preferences
1,INT-va1appis15-1504373018548-41332,My Wally receiver has lost Satellite signal in...,comp_part_signal_loss-issue,comp_part_signal_loss-issue
2,INT-sv1appis12-1503954587819-263368,I need a payment extension so i don't get my s...,payment_extension-request,payment_extension-request
3,8ed8e9b1-10f5-438d-c409-e616c3ff9ede,how can i find my local channels. it seems i d...,channel_package-issue,channel_package-issue
4,INT-sv1appis13-1504099917638-293735,Wanted to speak with someone about my bill,representative-request,representative-request


In [89]:
df_train,df_val,y_train,y_val = train_test_split(df,df['granular_intent'],train_size=0.8,random_state=42,stratify=df['granular_intent'])

In [90]:
# main_exp_folder = '/space/users/jgeorge/projects/k/tensorflow2-question-answering/input/dish/data/jan17_2020/'
main_exp_folder = '/var/extra/users/jgeorge/tf2.0/input/dish/data/jan17_2020/'

In [91]:
train_csv_file='/var/extra/users/jgeorge/tf2.0/input/dish/data/jan17_2020/train.txt'
eval_csv_file='/var/extra/users/jgeorge/tf2.0/input/dish/data/jan17_2020/val.txt'

df_train.to_csv(train_csv_file,sep='\t',header=False,index=False)
df_val.to_csv(eval_csv_file,sep='\t',header=False,index=False)

In [92]:
intent_file = '/var/extra/users/jgeorge/tf2.0/input/dish/data/jan17_2020/intentlist.txt'

In [93]:
with open(intent_file,'w',encoding='utf-8') as out_f:
    for intent in sorted(df_train['granular_intent'].unique()):
        out_f.write(intent+'\n')


In [13]:
intent_list = []
with open(intent_file,'r') as inp_f:
    for intent in inp_f:
        intent_list.append(intent.strip())

In [14]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    print("type constant ",type(tf.constant(0)))
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [15]:
def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [16]:
def _int64_feature_list(values):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=values))

In [17]:
train_data_path=os.path.join(main_exp_folder,'train.tfrecords')
eval_data_path=os.path.join(main_exp_folder,'eval.tfrecords')


In [18]:
class InputExample(object):
    def __init__(self,text,label):
        self.text = text
        self.label = label

In [19]:
class InputFeatures(object):
    def __init__(self,input_ids,input_mask,
                segment_ids,label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id


In [20]:
def convert_single_example(ex_index, example, label_list, max_seq_length,sentence_piece):
    CLS_ID = 2
    SEP_ID = 3
    # -2 for [CLS], [SEP]
    max_word_ids = max_seq_length # -2
    input_ids = []
    segment_ids = []
    input_mask = []
#     input_ids.append(CLS_ID)
    word_ids = sentence_piece.encode_as_ids(example.text)
    word_ids = word_ids[:max_word_ids]
    input_ids.extend(word_ids)
#     input_ids.append(SEP_ID)
    segment_ids = [0]*len(input_ids)
    input_mask = [1]*len(input_ids)
    if len(input_ids)<max_seq_length:
        diff = max_seq_length - len(input_ids)
        input_ids.extend([0]*diff)
        segment_ids.extend([0]*diff)
        input_mask.extend([0]*diff)
    assert(len(input_ids)==max_seq_length)
    assert(len(segment_ids)==max_seq_length)
    assert(len(input_mask)==max_seq_length)
    label_map = {label:i for i,label in enumerate(label_list)}
    label_id = label_map[example.label]
    feature = InputFeatures(input_ids,input_mask,segment_ids,label_id)
    return feature


In [21]:
max_seq_length = 128
# max_seq_length = bert_config.max_position_embeddings

In [22]:
albert_model_dir = '/var/extra/users/jgeorge/tf2.0/input/albert_base'

In [23]:
model_dir = albert_model_dir

In [24]:
albert_config='/var/extra/users/jgeorge/tf2.0/input/albert_base/assets/albert_config.json'

In [25]:
# bert_config = modeling.AlbertConfig.from_json_file(FLAGS.bert_config_file)

bert_config = modeling.AlbertConfig.from_json_file(albert_config)

In [26]:
bert_config.to_dict()

{'vocab_size': 30000,
 'hidden_size': 768,
 'num_hidden_layers': 12,
 'num_attention_heads': 12,
 'hidden_act': 'gelu',
 'intermediate_size': 3072,
 'hidden_dropout_prob': 0,
 'attention_probs_dropout_prob': 0,
 'max_position_embeddings': 512,
 'type_vocab_size': 2,
 'initializer_range': 0.02,
 'backward_compatible': True,
 'embedding_size': 128,
 'num_hidden_groups': 1,
 'net_structure_type': 0,
 'gap_size': 0,
 'num_memory_blocks': 0,
 'inner_group_num': 1,
 'down_scale_factor': 1}

In [27]:
def write_training_data(input_csv_file,output_file,delimiter='\t',max_seq_length=128):
    spm_model = os.path.join(model_dir, "assets", "30k-clean.model")
    sp = spm.SentencePieceProcessor()
    sp.load(spm_model)
   
    with open(input_csv_file,'r',encoding='utf-8') as csv_file, tf.io.TFRecordWriter(output_file) as tf_record_writer:
        #tried using binary format for bytes_list, but csv_reader requires text format
#     with open(input_csv_file,'rb') as csv_file, tf.io.TFRecordWriter(output_file) as tf_record_writer:
        csv_reader = csv.reader(csv_file,delimiter=delimiter,quotechar='"')
        for i,cols in enumerate(csv_reader):
            ##skipping filename & granular tag (granular tag & final tag are the same here)
#             yield cols[1:-1]
            features = collections.OrderedDict()
            
            filename = cols[0]
            text = cols[1]
            if(len(cols)<4):
                print('filename ',filename)
            intent = cols[2]
            input_example = InputExample(text,intent)
            feature = convert_single_example(ex_index=i,example=input_example,label_list=intent_list,
                                             max_seq_length=max_seq_length,sentence_piece=sp)
#             text_encoded = sp.encode_as_ids(text)
#             features['filename'] = _bytes_feature(filename.encode())
#             features['text'] = _int64_feature_list(text_encoded)
#             features['intent']  = _bytes_feature(intent.encode())

            features["input_ids"] = _int64_feature_list(feature.input_ids)
            features["input_mask"] = _int64_feature_list(feature.input_mask)
            features["segment_ids"] = _int64_feature_list(feature.segment_ids)
            features["label_id"] = _int64_feature(feature.label_id)
#             features["is_real_example"]
            tf_example = tf.train.Example(features=tf.train.Features(feature=features))
            tf_record_writer.write(tf_example.SerializeToString())

#     tf_record_writer.close()
write_training_data(train_csv_file,train_data_path,max_seq_length=max_seq_length)
write_training_data(eval_csv_file,eval_data_path,max_seq_length=max_seq_length)


In [28]:
raw_ds = tf.data.TFRecordDataset(train_data_path)

In [29]:
def _decode_data(record):
#     features = {
#         'filename':tf.io.FixedLenFeature([],tf.string),
#         'text':tf.io.VarLenFeature(tf.int64),
#         'intent':tf.io.FixedLenFeature([],tf.string)
#     }
    features = {
        "input_ids":tf.io.FixedLenFeature([max_seq_length],tf.int64),
        "input_mask":tf.io.FixedLenFeature([max_seq_length],tf.int64),
        "segment_ids":tf.io.FixedLenFeature([max_seq_length],tf.int64),
        "label_id":tf.io.FixedLenFeature([],tf.int64)
    }
    return tf.io.parse_single_example(record,features=features)


In [30]:
processed_data = raw_ds.map(_decode_data)

In [34]:
for line in processed_data.take(2):
    print(line)

{'input_ids': <tf.Tensor: shape=(128,), dtype=int64, numpy=
array([  483,   144,    51,  1071,  1839,    34, 14737,    37,   236,
         818,  1071,    60,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,   

In [35]:
albert_hub_url = "https://tfhub.dev/google/albert_base/2"

In [36]:
os.environ["TFHUB_CACHE_DIR"] = '/var/extra/engineering/tfhub_modules'

In [37]:
# strategy = distribution_utils.get_distribution_strategy('mirrored',num_gpus=1)

# strategy = tf.distribute.OneDeviceStrategy("device:GPU:2")
#since devices to use is set to 2 already, only 1 device is visible which is 0
strategy = tf.distribute.OneDeviceStrategy("device:GPU:0")

In [38]:
def get_loss_fn(num_classes, loss_factor=1.0):
  """Gets the classification loss function."""

  def classification_loss_fn(labels, logits):
    """Classification loss."""
    labels = tf.squeeze(labels)
    log_probs = tf.nn.log_softmax(logits, axis=-1)
    one_hot_labels = tf.one_hot(
        tf.cast(labels, dtype=tf.int32), depth=num_classes, dtype=tf.float32)
    per_example_loss = -tf.reduce_sum(
        tf.cast(one_hot_labels, dtype=tf.float32) * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    loss *= loss_factor
    return loss
  return classification_loss_fn  


In [39]:
albert_hub_url3='https://tfhub.dev/google/albert_base/3'

In [40]:
print(albert_hub_url)

https://tfhub.dev/google/albert_base/2


In [41]:
tags = set()
tags.add('train')
bert_module = hub.load(albert_hub_url3,tags=tags)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [42]:
bert_module.signatures['tokens']

In [43]:
# @tf.function
def get_model_part1(bert_config,float_type,num_labels,max_seq_length,final_layer_initializer=None,hub_module_url=None):
#     if final_layer_initializer is not None:  
#         initializer = final_layer_initializer
#     else:
#         initializer = tf.keras.initializers.TruncatedNormal(stddev=bert_config.initializer_range)

    if not hub_module_url:
        #TODO
        print("create the model")
        return None
#     input_word_ids = tf.keras.Input(shape=(max_seq_length),dtype=tf.int32,name='input_word_ids')
    input_ids = tf.keras.Input(shape=(max_seq_length),dtype=tf.int32,name='input_ids')
    input_mask = tf.keras.Input(shape=(max_seq_length),dtype=tf.int32,name='input_mask')
#     input_type_ids = tf.keras.Input(shape=(max_seq_length),dtype=tf.int32,name='input_type_ids')
    segment_ids = tf.keras.Input(shape=(max_seq_length),dtype=tf.int32,name='segment_ids')
    return input_ids,input_mask,segment_ids

In [44]:

def get_model_bert(bert_config,float_type,num_labels,max_seq_length,final_layer_initializer,hub_module_url,
                   input_ids,input_mask,segment_ids):
    tags = set()
    tags.add('train')
#     bert_module = hub.Module(hub_module_url,trainable=True)
    bert_module = hub.load(hub_module_url,tags=tags)
    bert_inputs = {'input_ids':input_ids,
                   'input_mask':input_mask,
                   'segment_ids':segment_ids}
    # https://www.tensorflow.org/hub/common_issues
#     bert_outputs = bert_module.signatures['tokens'](
#       inputs=bert_inputs,
#       signature="tokens",
#       as_dict=True)

#     bert_outputs = bert_module.signatures['tokens'](bert_inputs) #bert_outputs
#     bert_outputs = bert_module.signatures['tokens'](input_ids=input_ids,input_mask=input_mask,segment_ids=segment_ids)
#     print(bert_outputs)
    
#     bert_model = hub.KerasLayer(albert_hub_url,trainable=True,tags=tags,signature='tokens',signature_outputs_as_dict=True)
    
    ### pooled_output will give the representation for [CLS]
    ### sequence_output will give representations for all tokens
    bert_model = hub.KerasLayer(albert_hub_url,trainable=True,tags=tags,signature='tokens',output_key='pooled_output')
    print(' bert_model ',bert_model)
#     pooled_output,_ = bert_model([input_word_ids,input_mask,input_type_ids]) 
#     pooled_output,_ = bert_model(input_ids=input_ids,input_mask=input_mask,segment_ids=segment_ids) 
#     pooled_output,_ = bert_model(inputs = [input_ids,input_mask,segment_ids]) 
#     pooled_output,temp = bert_model(inputs = {'input_ids':input_ids,'input_mask':input_mask,'segment_ids':segment_ids})
    pooled_output = bert_model(inputs = {'input_ids':input_ids,'input_mask':input_mask,'segment_ids':segment_ids})
#     output = tf.keras.layers.Dropout(rate = bert_config.hidden_dropout_prob)(pooled_output)
#     pooled_output = bert_outputs['pooled_output']
    output = pooled_output
#     print('pooled_output ',pooled_output,' temp '+temp)
    print('pooled_output ',pooled_output)
#     output = bert_outputs['pooled_output']
    return output,bert_model

In [45]:
# @tf.function
def get_model_part3(bert_output,num_labels,final_layer_initializer):
    if final_layer_initializer is not None:  
        initializer = final_layer_initializer
    else:
        initializer = tf.keras.initializers.TruncatedNormal(stddev=bert_config.initializer_range)
    #     output = tf.keras.layers.Dropout(rate = bert_config.hidden_dropout_prob)(pooled_output)
    output = tf.keras.layers.Dense(num_labels,kernel_initializer=initializer,name='output',dtype=tf.float32)(bert_output)
    return output

In [46]:

def combine_model(bert_config,float_type,num_labels,max_seq_length,final_layer_initializer=None,hub_module_url=None):
    input_ids,input_mask,segment_ids = get_model_part1(bert_config,float_type,num_labels,max_seq_length,final_layer_initializer,hub_module_url)
    bert_output,bert_model = get_model_bert(bert_config,float_type,num_labels,max_seq_length,final_layer_initializer,hub_module_url,input_ids,input_mask,segment_ids)
    print('bert_output',bert_output)
    output = get_model_part3(bert_output,num_labels,final_layer_initializer)
    return tf.keras.Model(inputs={'input_ids':input_ids,
                                 'input_mask':input_mask,
                                 'segment_ids':segment_ids},
                         outputs=output),bert_model

In [47]:
# @tf.function
def get_classifier_model(bert_config,float_type,num_labels,max_seq_length,final_layer_initializer=None,hub_module_url=None):
    if final_layer_initializer is not None:  
        initializer = final_layer_initializer
    else:
        initializer = tf.keras.initializers.TruncatedNormal(stddev=bert_config.initializer_range)

    if not hub_module_url:
        #TODO
        print("create the model")
        return None
#     input_word_ids = tf.keras.Input(shape=(max_seq_length),dtype=tf.int32,name='input_word_ids')
    input_ids = tf.keras.Input(shape=(max_seq_length),dtype=tf.int32,name='input_ids')
    input_mask = tf.keras.Input(shape=(max_seq_length),dtype=tf.int32,name='input_mask')
#     input_type_ids = tf.keras.Input(shape=(max_seq_length),dtype=tf.int32,name='input_type_ids')
    segment_ids = tf.keras.Input(shape=(max_seq_length),dtype=tf.int32,name='segment_ids')
    tags = set()
    tags.add('train')
#     bert_module = hub.Module(hub_module_url,trainable=True)
    bert_module = hub.load(hub_module_url,tags=tags)
    bert_inputs = {'input_ids':input_ids,
                   'input_mask':input_mask,
                   'segment_ids':segment_ids}
    # https://www.tensorflow.org/hub/common_issues
#     bert_outputs = bert_module.signatures['tokens'](
#       inputs=bert_inputs,
#       signature="tokens",
#       as_dict=True)

#     bert_outputs = bert_module.signatures['tokens'](
#       bert_inputs)
    bert_outputs = bert_module.signatures['tokens'](input_ids=input_ids,input_mask=input_mask,segment_ids=segment_ids)
    print(bert_outputs)
    
#     bert_model = hub.KerasLayer(albert_hub_url,trainable=True,tags=tags,signature='tokens',signature_outputs_as_dict=True)
#     pooled_output,_ = bert_model([input_word_ids,input_mask,input_type_ids]) 
#     pooled_output,_ = bert_model([segment_ids,input_ids,input_mask]) 
#     output = tf.keras.layers.Dropout(rate = bert_config.hidden_dropout_prob)(pooled_output)
#     pooled_output = bert_outputs['pooled_output']
    output = bert_outputs['pooled_output']
#     output = tf.keras.layers.Dropout(rate = bert_config.hidden_dropout_prob)(pooled_output)
    output = tf.keras.layers.Dense(num_labels,kernel_initializer=initializer,name='output',dtype=tf.float32)(output)
    return tf.keras.Model(inputs={'input_ids':input_ids,
                                 'input_mask':input_mask,
                                 'segment_ids':segment_ids},
                         outputs=output),bert_model
    

In [330]:
#clear the existing tensorflow graph
tf.keras.backend.clear_session()

In [48]:
def run_bert_classifier(strategy,
                        bert_config,
                        input_meta_data,
                        model_dir,
                        epochs,
                        steps_per_epoch,
                        steps_per_loop,
                        eval_steps,
                        warmup_steps,
                        initial_lr,
                        init_checkpoint,
                        train_input_fn,
                        eval_input_fn,
                        custom_callbacks=None,
                        run_eagerly=False,
                        use_keras_compile_fit=False):
    """Run BERT classifier training using low-level API."""
    max_seq_length = input_meta_data['max_seq_length']
    num_classes = input_meta_data['num_labels']
    def _get_classifier_model():
#     bert_models.classifier_model
#     classifier_model,core_model = get_classifier_model(bert_config=bert_config,
#                                                                float_type=tf.float32,
#                                                               num_labels=num_classes,
#                                                               max_seq_length=max_seq_length,
#                                                                hub_module_url=albert_hub_url)
        classifier_model,core_model = combine_model(bert_config=bert_config,
                                                                   float_type=tf.float32,
                                                                  num_labels=num_classes,
                                                                  max_seq_length=max_seq_length,
                                                                   hub_module_url=albert_hub_url)
        classifier_model.optimizer = optimization.create_optimizer(init_lr=initial_lr,
                                                                   num_train_steps=steps_per_epoch*epochs,
                                                                   num_warmup_steps=warmup_steps)
        return classifier_model,core_model
    loss_multiplier = 1
    loss_fn = get_loss_fn(num_classes,loss_multiplier)
    
    def metric_fn():
        return tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy',dtype=tf.float32)
    return model_training_utils.run_customized_training_loop(
      strategy=strategy,
      model_fn=_get_classifier_model,
      loss_fn=loss_fn,
      model_dir=model_dir,
      steps_per_epoch=steps_per_epoch,
      steps_per_loop=steps_per_loop,
      epochs=epochs,
      train_input_fn=train_input_fn,
      eval_input_fn=eval_input_fn,
      eval_steps=eval_steps,
      init_checkpoint=init_checkpoint,
      metric_fn=metric_fn,
      custom_callbacks=custom_callbacks,
      run_eagerly=run_eagerly)


In [49]:
albert_model_dir

'/var/extra/users/jgeorge/tf2.0/input/albert_base'

In [ ]:
strategy.experimental_run_v2()

In [51]:
input_meta_data = {
    'max_seq_length':128,
    'num_labels':142,
    'train_data_size':20574,
    'eval_data_size':5144
}

In [52]:
input_meta_path = os.path.join(main_exp_folder,'input_metadata.json')

In [53]:
#writing metadata file
with open(input_meta_path,'w',encoding='utf-8') as jf:
    jf.write(json.dumps(input_meta_data))

In [54]:
#reading metadata file
with open(input_meta_path,'r',encoding='utf-8') as jf:
#     input_meta_data = json.loads(jf.read())
    input_meta_data = json.load(jf)

In [56]:
steps_per_loop = 1
learning_rate=1e-5
epochs=1

In [57]:
train_batch_size=16
eval_batch_size=16

In [58]:
model_dir_dish=os.path.join(main_exp_folder,'models')


In [59]:
init_checkpoint=None

In [60]:
train_data_size = input_meta_data['train_data_size']
steps_per_epoch = int(train_data_size / train_batch_size)
print(steps_per_epoch)

1285


In [61]:
warmup_steps = int(epochs * train_data_size * 0.1 / train_batch_size)
eval_steps = int(math.ceil(input_meta_data['eval_data_size'] / eval_batch_size))

In [63]:
def decode_record(record, name_to_features):
  """Decodes a record to a TensorFlow example."""
  example = tf.io.parse_single_example(record, name_to_features)

  # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
  # So cast all int64 to int32.
  for name in list(example.keys()):
    t = example[name]
    if t.dtype == tf.int64:
      t = tf.cast(t, tf.int32)
    example[name] = t

  return example


In [64]:
def single_file_dataset(input_file, name_to_features):
  """Creates a single-file dataset to be passed for BERT custom training."""
  # For training, we want a lot of parallel reading and shuffling.
  # For eval, we want no shuffling and parallel reading doesn't matter.
  d = tf.data.TFRecordDataset(input_file)
  d = d.map(lambda record: decode_record(record, name_to_features))

  # When `input_file` is a path to a single file or a list
  # containing a single path, disable auto sharding so that
  # same input file is sent to all workers.
  if isinstance(input_file, str) or len(input_file) == 1:
    options = tf.data.Options()
    options.experimental_distribute.auto_shard_policy = (
        tf.data.experimental.AutoShardPolicy.OFF)
    d = d.with_options(options)
  return d


In [65]:
def create_classifier_dataset(file_path,
                              seq_length,
                              batch_size,
                              is_training=True,
                              input_pipeline_context=None):
  """Creates input dataset from (tf)records files for train/eval."""
  name_to_features = {
      'input_ids': tf.io.FixedLenFeature([seq_length], tf.int64),
      'input_mask': tf.io.FixedLenFeature([seq_length], tf.int64),
      'segment_ids': tf.io.FixedLenFeature([seq_length], tf.int64),
      'label_id': tf.io.FixedLenFeature([], tf.int64)
#       'is_real_example': tf.io.FixedLenFeature([], tf.int64),
  }
  dataset = single_file_dataset(file_path, name_to_features)

  # The dataset is always sharded by number of hosts.
  # num_input_pipelines is the number of hosts rather than number of cores.
  if input_pipeline_context and input_pipeline_context.num_input_pipelines > 1:
    dataset = dataset.shard(input_pipeline_context.num_input_pipelines,
                            input_pipeline_context.input_pipeline_id)

  def _select_data_from_record(record):
#     x = {
#         'input_word_ids': record['input_ids'],
#         'input_mask': record['input_mask'],
#         'input_type_ids': record['segment_ids']
#     }
    x = {
        'input_ids': record['input_ids'],
        'input_mask': record['input_mask'],
        'segment_ids': record['segment_ids']
    }
    y = record['label_id']
    return (x, y)

  dataset = dataset.map(_select_data_from_record)

  if is_training:
    dataset = dataset.shuffle(100)
    dataset = dataset.repeat()

  dataset = dataset.batch(batch_size, drop_remainder=is_training)
  dataset = dataset.prefetch(1024)
  return dataset


In [66]:
def get_dataset_fn(input_file_pattern, max_seq_length, global_batch_size,
                   is_training):
  """Gets a closure to create a dataset."""
  def _dataset_fn(ctx=None):
    """Returns tf.data.Dataset for distributed BERT pretraining."""
    batch_size = ctx.get_per_replica_batch_size(
        global_batch_size) if ctx else global_batch_size
#     dataset = input_pipeline.create_classifier_dataset(
    dataset = create_classifier_dataset(
        input_file_pattern,
        max_seq_length,
        batch_size,
        is_training=is_training,
        input_pipeline_context=ctx)
    return dataset
  return _dataset_fn


In [67]:
train_input_fn = get_dataset_fn(
#       FLAGS.train_data_path,
      train_data_path,
      max_seq_length,
#       FLAGS.train_batch_size,
      train_batch_size,
      is_training=True)
eval_input_fn = get_dataset_fn(
#       FLAGS.eval_data_path,
      eval_data_path,
      max_seq_length,
#       FLAGS.eval_batch_size,
      eval_batch_size,
      is_training=False)


In [68]:
trained_model = run_bert_classifier(
      strategy,
      bert_config,
      input_meta_data,
      albert_model_dir,
      epochs,
      steps_per_epoch,
      steps_per_loop,
      eval_steps,
      warmup_steps,
      learning_rate,
      init_checkpoint,
      train_input_fn,
      eval_input_fn,
      run_eagerly=False,
      use_keras_compile_fit=False)


 bert_model  <tensorflow_hub.keras_layer.KerasLayer object at 0x7f1ed83b7610>
pooled_output  Tensor("keras_layer/Identity:0", shape=(None, 768), dtype=float32)
bert_output Tensor("keras_layer/Identity:0", shape=(None, 768), dtype=float32)


In [ ]:
trained_model.save()

In [69]:
trained_model.evaluate(eval_input_fn)

ValueError: Failed to find data adapter that can handle input: <class 'function'>, <class 'NoneType'>

In [72]:
def metric_fn():
    return tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy',dtype=tf.float32)

In [73]:
metric = metric_fn()

In [70]:
def evaluate(eval_input_fn,model):
    eval_iter = iter(strategy.experimental_distribute_datasets_from_function(eval_input_fn))
    
    def _test_step_fn(inputs,label):
#         inputs,label = inputs
        model_outputs = model(inputs,training=False)
        metric.update_state(label,model_outputs)
    strategy.experimental_run_v2(_test_step_fn,args=(next(eval_iter)))

In [74]:
evaluate(eval_input_fn,trained_model)

In [78]:
metric.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.125>

In [365]:
model_export_dir = '/var/extra/users/jgeorge/tf2.0/input/albert_base_custom/'

In [368]:
spm_model = os.path.join(model_dir, "assets", "30k-clean.model")
sp = spm.SentencePieceProcessor()
sp.load(spm_model)

True

In [369]:
trained_model.sp_model_file = tf.saved_model.Asset(spm_model)

In [373]:
trained_model.sp_model_file

In [374]:
trained_model.save(model_export_dir,include_optimizer=False,save_format='tf')

INFO:tensorflow:Assets written to: /var/extra/users/jgeorge/tf2.0/input/albert_base_custom/assets


INFO:tensorflow:Assets written to: /var/extra/users/jgeorge/tf2.0/input/albert_base_custom/assets


In [364]:
metric.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.125>

In [345]:
bert_model = hub.KerasLayer(albert_hub_url,trainable=True,)

ValueError: Unknown signature default in https://tfhub.dev/google/albert_base/2 (available signatures: _SignatureMap({'tokenization_info': <tensorflow.python.eager.wrap_function.WrappedFunction object at 0x7f550be41450>, 'mlm': <tensorflow.python.eager.wrap_function.WrappedFunction object at 0x7f57ec038a50>, 'tokens': <tensorflow.python.eager.wrap_function.WrappedFunction object at 0x7f55086cf990>})).

In [94]:
bert_model = hub.KerasLayer(albert_hub_url,trainable=True,signature='tokens')

In [ ]:
bert_model()

In [126]:
trained_model = run_bert_classifier(
      strategy,
      bert_config,
      input_meta_data,
      albert_model_dir,
      epochs,
      steps_per_epoch,
      steps_per_loop,
      eval_steps,
      warmup_steps,
      learning_rate,
      init_checkpoint,
      train_input_fn,
      eval_input_fn,
      run_eagerly=False,
      use_keras_compile_fit=False)


TypeError: in converted code:

    /opt/custom/python/anaconda3/envs/tf2.1/lib/python3.7/site-packages/tensorflow_hub/keras_layer.py:209 call  *
        result = f()
    /opt/custom/python/anaconda3/envs/tf2.1/lib/python3.7/site-packages/tensorflow_core/python/eager/function.py:1551 __call__  *
        return self._call_impl(args, kwargs)
    /opt/custom/python/anaconda3/envs/tf2.1/lib/python3.7/site-packages/tensorflow_core/python/eager/function.py:1570 _call_impl
        ).format(self._num_positional_args, self._arg_keywords, args))

    TypeError: Expected at most 0 positional arguments (and the rest keywords, of ['segment_ids', 'input_ids', 'input_mask']), got ([<tf.Tensor 'segment_ids:0' shape=(None, 128) dtype=int32>, <tf.Tensor 'input_ids:0' shape=(None, 128) dtype=int32>, <tf.Tensor 'input_mask:0' shape=(None, 128) dtype=int32>],). When calling a concrete function, positional arguments may not be bound to Tensors within nested structures.


In [100]:
tags = set()
tags.add('train')

loaded_albert_model = tf.saved_model.load(albert_model_dir,tags=tags)

In [106]:
loaded_albert_model.signatures

_SignatureMap({'mlm': <tensorflow.python.eager.wrap_function.WrappedFunction object at 0x7f551de1d9d0>, 'tokens': <tensorflow.python.eager.wrap_function.WrappedFunction object at 0x7f551d98e090>, 'tokenization_info': <tensorflow.python.eager.wrap_function.WrappedFunction object at 0x7f551d5d5050>})

In [105]:
loaded_albert_model.signatures['mlm']